In [1]:
!pip install selenium

In [20]:
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import requests

from bs4 import BeautifulSoup

Write a python program which searches all the product under a particular product from www.amazon.in. The
product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for
guitars

In [34]:
driver=webdriver.Chrome()

In [35]:
driver.get("https://www.amazon.com/")

In [41]:

product=driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')
product.send_keys('cellphone')

In [42]:
search=driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]')
search.click()

In [43]:
cellphone=[]
cellphone_tags=driver.find_elements(By.XPATH,'//span[@class="a-size-medium a-color-base a-text-normal"]')
for i in cellphone_tags:
    cellphone.append(i.text.split(",")[0].split("|")[0])

In [44]:
cellphone

['TCL 40XL 2023 Unlocked Cell Phone 6GB + 256GB',
 'Blackview Unlocked Android Phones',
 'ExAchat A14 Pro Max 5G Unlocked Smartphone 200MP Camera 7.3" FHD+ Display 120HzD 6GB+128GB/1TB Cell Phones Unlocked Expandable 8000mAh Dual SIM/GPS/Fingerprint/Face ID',
 'Motorola Moto G Stylus ',
 'Motorola Moto G Stylus 5G ',
 'Google Pixel 7a - Unlocked Android Cell Phone - Smartphone with Wide Angle Lens and 24-Hour Battery - 128 GB -  Sea',
 'Google Pixel 6a - 5G Android Phone - Unlocked Smartphone with 12 Megapixel Camera and 24-Hour Battery - Charcoal',
 'SAMSUNG Galaxy A14 5G + 4G LTE (128GB + 4GB) Unlocked Worldwide (Only T-Mobile/Mint/Tello USA Market) 1 Year Warranty Latin America 6.6" 50MP Triple Camera + (15W Wall Charger) (Silver)',
 'SAMSUNG Galaxy A54 5G A Series Cell Phone',
 'Nokia 6300 4G ',
 'OnePlus Nord N200 ',
 'ZXSWONLY Charging Station for Multiple Devices',
 'SAMSUNG Galaxy A03 Core (SM-A032/DS) Dual SIM 32GB/ 2GB RAM',
 'SAMSUNG Galaxy Z Fold 5 Cell Phone + Storage Upgr

In [56]:
start_page = 0
end_page = 3
urls = []
for page in range(start_page,end_page+1):
    try:
        page_urls = driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-no-outline"]')
        
        
        for url in page_urls:
            url = url.get_attribute('href')     
            if url[0:4]=='http':                
                urls.append(url)                
        print("Product urls of page {} has been scraped.".format(page+1))
        
         
        nxt_button = driver.find_element(By.XPATH,'/html/body/div[1]/div[2]/div[1]/div[1]/div/span[1]/div[1]/div[29]/div/div/span/a[3]')    
        if nxt_button.text == 'Next→':                                            
            nxt_button.click()                                                  
                                                                    
           
        elif driver.find_element(By.XPATH,'/html/body/div[1]/div[2]/div[1]/div[1]/div/span[1]/div[1]/div[29]/div/div/span/a[3]').text == 'Next→':    
            print("No new pages exist. Breaking the loop") 
            break
        
        
       
            
    except StaleElementReferenceException as e:               
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        
        driver.get(next_page)

Product urls of page 1 has been scraped.
Product urls of page 2 has been scraped.
Product urls of page 3 has been scraped.
Product urls of page 4 has been scraped.


In [57]:
import pandas as pd
prod = {}
prod['Brand']=[]
prod['Name']=[]
prod['Rating']=[]
prod['No. of ratings']=[]
prod['Price']=[]
prod['Return/Exchange']=[]
prod['Expected Delivery']=[] 
prod['Availability']=[]
prod['Other Details']=[]
prod['URL']=[]

In [58]:
for url in urls[:4]:
    driver.get(url)                                                        
    print("Scraping URL = ", url)
    
    
    try:
        brand = driver.find_element(By.XPATH,'//a[@id="bylineInfo"]')      
        prod['Brand'].append(brand.text)
    except NoSuchElementException:
        prod['Brand'].append('-')
    
    try:
        name = driver.find_element(By.XPATH,'//h1[@id="title"]/span')      
        prod['Name'].append(name.text)
    except NoSuchElementException:
        prod['Name'].append('-')
    
    try:
        rating = driver.find_element(By.XPATH,'//span[@id="acrPopover"]')  
        prod['Rating'].append(rating.get_attribute("title"))
    except NoSuchElementException:
        prod['Rating'].append('-')
    
    try:
        n_rating = driver.find_element(By.XPATH,'//a[@id="acrCustomerReviewLink"]/span')     
        prod['No. of ratings'].append(n_rating.text)
    except NoSuchElementException:
        prod['No. of ratings'].append('-')
        
    try:
        price = driver.find_element(By.XPATH,'//span[@id="priceblock_ourprice"]')            
        prod['Price'].append(price.text)
    except NoSuchElementException:
        prod['Price'].append('-')
    try:                                                                                     
        ret = driver.find_element(By.XPATH,'//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        prod['Return/Exchange'].append(ret.text)
    except NoSuchElementException:
        prod['Return/Exchange'].append('-')
    try:
        delivry = driver.find_element(By.XPATH,'//div[@id="ddmDeliveryMessage"]/b')         
        prod['Expected Delivery'].append(delivry.text)
    except NoSuchElementException:
        prod['Expected Delivery'].append('-')
    
    try:
        avl = driver.find_element(By.XPATH,'//div[@id="availability"]/span')                
        prod['Availability'].append(avl.text)
    except NoSuchElementException:
        prod['Availability'].append('-')
    
    try:                                                                                    
        dtls = driver.find_element(By.XPATH,'//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        prod['Other Details'].append('  ||  '.join(dtls.text.split('\n')))
    except NoSuchElementException:
        prod['Other Details'].append('-')
        
    prod['URL'].append(url)

Scraping URL =  https://www.amazon.com/sspa/click?ie=UTF8&spc=MTo3MTAxNjg4NzI1NTY2NTE6MTY5MTU4ODIzODpzcF9hdGY6MjAwMTY5MDA1MTc0OTk4OjowOjo&url=%2FTCL-Unlocked-Display-Smartphone-Android%2Fdp%2FB0BTP83RM6%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dcellphone%26qid%3D1691588238%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1
Scraping URL =  https://www.amazon.com/sspa/click?ie=UTF8&spc=MTo3MTAxNjg4NzI1NTY2NTE6MTY5MTU4ODIzODpzcF9hdGY6MzAwMDI3NzU2NTE2ODAyOjowOjo&url=%2FBlackview-Unlocked-Android-Cellphone-Display%2Fdp%2FB0C6F4T2K9%2Fref%3Dsr_1_2_sspa%3Fkeywords%3Dcellphone%26qid%3D1691588238%26sr%3D8-2-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1
Scraping URL =  https://www.amazon.com/ExAchat-Pro-Smartphone-Expandable-Fingerprint/dp/B0C8T8XVLL/ref=sr_1_3?keywords=cellphone&qid=1691588238&sr=8-3
Scraping URL =  https://www.amazon.com/Stylus-battery-Unlocked-Motorola-Twilight/dp/B09PFC2DVD/ref=sr_1_4?keywords=cellphone&qid=1691588238&sr=8-4


In [59]:
df = pd.DataFrame.from_dict(prod)
df

,Brand,Name,Rating,No. of ratings,Price,Return/Exchange,Expected Delivery,Availability,Other Details,URL
0,Visit the TCL Store,"TCL 40XL 2023 Unlocked Cell Phone 6GB + 256GB,...",4.0 out of 5 stars,177 ratings,-,-,-,-,The TCL 40XL works with most major U.S. carrie...,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
1,Visit the Blackview Store,"Blackview Unlocked Android Phones, 2023 A53 Mo...",4.2 out of 5 stars,48 ratings,-,-,-,-,🚀【Up to 7GB RAM+32GB ROM/ 1TB Expand】🚀 4GB RAM...,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
2,Brand: ExAchat,ExAchat A14 Pro Max 5G Unlocked Smartphone 200...,4.5 out of 5 stars,2 ratings,-,-,-,-,★【Enjoy Larger Screen】This unlocked android ph...,https://www.amazon.com/ExAchat-Pro-Smartphone-...
3,Visit the Motorola Store,Motorola Moto G Stylus | 2022 | 2-Day battery ...,4.3 out of 5 stars,"1,614 ratings",-,-,-,Only 10 left in stock - order soon,"Carrier compatibility: AT&T: VoLTE, Verizon: V...",https://www.amazon.com/Stylus-battery-Unlocked...


In [60]:
#saving dataframe to csv file
df.to_csv('cellphone.csv')

Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’

In [2]:
driver=webdriver.Chrome()

Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


In [3]:
driver.get('https://images.google.com/')

In [8]:
search_bar = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea')    
search_bar.send_keys("fruits")       
search_button = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button/div')   
search_button.click() 


In [9]:
print("start scrolling to generate more images on the page...")
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")

start scrolling to generate more images on the page...


In [17]:

 images = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')



In [18]:
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

16

In [23]:
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response= requests.get(img_urls[i])
   

Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps. 


In [217]:
driver=webdriver.Chrome()

In [218]:
driver.get("https://maps.google.com/")

In [219]:
my_city=driver.find_element(By.XPATH,'/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div/div[2]/form/input')
my_city.send_keys('kolkata')

In [220]:
search=driver.find_element(By.CLASS_NAME,'pzfvzf')
search.click()

In [314]:

url= driver.current_url

print("URL Extracted: ", url)
list=url.split(",")
longitude=list[2]
latitude=list[1].split('@')[1]
print("latitude:",latitude,"longitude:",longitude)








    
    


URL Extracted:  https://www.google.com/maps/search/latitude+and+longitude+Kolkata,+West+Bengal/@22.5548078,88.1912327,11z/data=!3m1!4b1?entry=ttu
latitude: 22.5548078 longitude: 88.1912327


Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped:
“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”

In [363]:
driver=webdriver.Chrome()

In [364]:
driver.get("https://www.forbes.com/")

In [366]:
search=driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div[1]/div/div')
search.click()

In [367]:
bill=driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div[1]/div/div[2]/ul/li[2]')
bill.click()

In [368]:
search_btn=driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div[1]/div/div[2]/ul/li[2]/div[2]/div[3]/ul/li[1]')
search_btn.click()

In [464]:
first_worth=driver.find_element(By.XPATH,'/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[3]/div')
first_name=driver.find_element(By.XPATH,'/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[2]/div')
first_age=driver.find_element(By.XPATH,'/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[4]/div')
first_country=driver.find_element(By.XPATH,'/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[5]')
first_source=driver.find_element(By.XPATH,'/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[6]/div/span')
first_industry=driver.find_element(By.XPATH,'/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[7]/div')

In [378]:
rank=[]
title_tags=driver.find_elements(By.XPATH,'//div[@class="Table_rank___YBhk Table_dataCell__2QCve"]')
for i in title_tags:
    title=i.text
    rank.append(title)

In [379]:
rank

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '17',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '31',
 '33',
 '34',
 '35',
 '35',
 '37',
 '38',
 '39',
 '40',
 '41',
 '41',
 '43',
 '43',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '65',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '72',
 '74',
 '74',
 '76',
 '77',
 '77',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '84',
 '86',
 '86',
 '88',
 '89',
 '89',
 '89',
 '92',
 '93',
 '94',
 '94',
 '94',
 '97',
 '97',
 '99',
 '100',
 '101',
 '101',
 '103',
 '104',
 '104',
 '106',
 '107',
 '108',
 '108',
 '108',
 '108',
 '112',
 '113',
 '114',
 '115',
 '116',
 '116',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '124',
 '126',
 '127',
 '128',
 '128',
 '130',
 '130',
 '130',
 '133',
 '133',
 '133',
 '136',
 '137',
 '138',
 '13

In [452]:
name=[]
name.append(first_name.text)
title_tags=driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]')
for i in title_tags:
    title=i.text.split('\n')[1].split('\n')[0]
    name.append(title)

In [453]:
name

['Bernard Arnault & family',
 'Elon Musk',
 'Jeff Bezos',
 'Larry Ellison',
 'Warren Buffett',
 'Bill Gates',
 'Michael Bloomberg',
 'Carlos Slim Helu & family',
 'Mukesh Ambani',
 'Steve Ballmer',
 'Francoise Bettencourt Meyers & family',
 'Larry Page',
 'Amancio Ortega',
 'Sergey Brin',
 'Zhong Shanshan',
 'Mark Zuckerberg',
 'Charles Koch',
 'Julia Koch & family',
 'Jim Walton',
 'Rob Walton',
 'Alice Walton',
 'David Thomson & family',
 'Michael Dell',
 'Gautam Adani',
 'Phil Knight & family',
 'Zhang Yiming',
 'Dieter Schwarz',
 'François Pinault & family',
 'Klaus-Michael Kuehne',
 'Giovanni Ferrero',
 'Jacqueline Mars',
 'John Mars',
 'Li Ka-shing',
 'Ma Huateng',
 'Miriam Adelson & family',
 'Ken Griffin',
 'Mark Mateschitz',
 'Robin Zeng',
 'Tadashi Yanai & family',
 'Len Blavatnik',
 'Alain Wertheimer',
 'Gerard Wertheimer',
 'Gianluigi Aponte',
 'Rafaela Aponte-Diamant',
 'Colin Zheng Huang',
 'Reinhold Wuerth & family',
 'Lee Shau Kee',
 'Jeff Yass',
 'Jim Simons',
 'Stephe

In [454]:
net_worth=[]
net_worth.append(first_worth.text)
title_tags=driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]')
for i in title_tags:
    title=i.text.split('\n')[2]
    net_worth.append(title)

In [455]:
net_worth

['$211 B',
 '$180 B',
 '$114 B',
 '$107 B',
 '$106 B',
 '$104 B',
 '$94.5 B',
 '$93 B',
 '$83.4 B',
 '$80.7 B',
 '$80.5 B',
 '$79.2 B',
 '$77.3 B',
 '$76 B',
 '$68 B',
 '$64.4 B',
 '$59 B',
 '$59 B',
 '$58.8 B',
 '$57.6 B',
 '$56.7 B',
 '$54.4 B',
 '$50.1 B',
 '$47.2 B',
 '$45.1 B',
 '$45 B',
 '$42.9 B',
 '$40.1 B',
 '$39.1 B',
 '$38.9 B',
 '$38.3 B',
 '$38.3 B',
 '$38 B',
 '$35.3 B',
 '$35 B',
 '$35 B',
 '$34.7 B',
 '$33.4 B',
 '$32.6 B',
 '$32.1 B',
 '$31.6 B',
 '$31.6 B',
 '$31.2 B',
 '$31.2 B',
 '$30.2 B',
 '$29.7 B',
 '$29.5 B',
 '$28.5 B',
 '$28.1 B',
 '$27.8 B',
 '$27.4 B',
 '$27 B',
 '$26.7 B',
 '$26.6 B',
 '$25.6 B',
 '$25.5 B',
 '$25.3 B',
 '$25.2 B',
 '$24.6 B',
 '$24.4 B',
 '$24.2 B',
 '$23.7 B',
 '$23.5 B',
 '$23.4 B',
 '$23.1 B',
 '$23.1 B',
 '$22.9 B',
 '$22.6 B',
 '$22.4 B',
 '$22.1 B',
 '$22 B',
 '$21.6 B',
 '$21.6 B',
 '$21.2 B',
 '$21.2 B',
 '$21.1 B',
 '$21 B',
 '$21 B',
 '$20.9 B',
 '$20.5 B',
 '$20.2 B',
 '$19.6 B',
 '$19.1 B',
 '$19 B',
 '$19 B',
 '$18.9 B',
 '$1

In [456]:
age=[]
age.append(first_age.text)
title_tags=driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]')
for i in title_tags:
    title=i.text.split('\n')[3]
    age.append(title)

In [457]:
age

['74',
 '51',
 '59',
 '78',
 '92',
 '67',
 '81',
 '83',
 '65',
 '67',
 '69',
 '50',
 '87',
 '49',
 '68',
 '38',
 '87',
 '60',
 '74',
 '78',
 '73',
 '65',
 '58',
 '60',
 '85',
 '39',
 '83',
 '86',
 '85',
 '58',
 '83',
 '87',
 '94',
 '51',
 '77',
 '54',
 '30',
 '54',
 '74',
 '65',
 '74',
 '72',
 '82',
 '78',
 '43',
 '87',
 '95',
 '64',
 '84',
 '76',
 '60',
 '69',
 '51',
 '69',
 '77',
 '74',
 '78',
 '51',
 '56',
 '52',
 '82',
 '62',
 '58',
 '80',
 '80',
 '83',
 '70',
 '81',
 '65',
 '66',
 '52',
 '61',
 '67',
 '36',
 '52',
 '60',
 '90',
 '77',
 '57',
 '72',
 '84',
 '61',
 '73',
 '59',
 '55',
 '72',
 '57',
 '57',
 '77',
 '65',
 '70',
 '61',
 '72',
 '66',
 '87',
 '73',
 '90',
 '83',
 '92',
 '70',
 '72',
 '55',
 '72',
 '75',
 '67',
 '55',
 '67',
 'N/A',
 'N/A',
 '83',
 '76',
 '67',
 '45',
 '68',
 '56',
 '83',
 '81',
 '78',
 '68',
 '56',
 '59',
 '69',
 '95',
 '55',
 '65',
 '93',
 '67',
 '57',
 '94',
 '78',
 '76',
 '47',
 '80',
 '80',
 '71',
 '60',
 '90',
 '64',
 '75',
 '58',
 '57',
 '50',
 '69

In [458]:
country=[]
country.append(first_country.text)
title_tags=driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]')
for i in title_tags:
    title=i.text.split('\n')[4]
    country.append(title)

In [459]:
country

['France',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'Mexico',
 'India',
 'United States',
 'France',
 'United States',
 'Spain',
 'United States',
 'China',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'Canada',
 'United States',
 'India',
 'United States',
 'China',
 'Germany',
 'France',
 'Germany',
 'Italy',
 'United States',
 'United States',
 'Hong Kong',
 'China',
 'United States',
 'United States',
 'Austria',
 'Hong Kong',
 'Japan',
 'United States',
 'France',
 'France',
 'Switzerland',
 'Switzerland',
 'China',
 'Germany',
 'Hong Kong',
 'United States',
 'United States',
 'United States',
 'Germany',
 'Australia',
 'China',
 'Mexico',
 'India',
 'Indonesia',
 'United States',
 'Russia',
 'Germany',
 'United States',
 'Indonesia',
 'Russia',
 'China',
 'China',
 'Chile',
 'Indonesia',
 'United Kingdom',
 'India',
 'Japan',
 'Russia',
 'France',
 

In [460]:
source=[]
source.append(first_source.text)
title_tags=driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]')
for i in title_tags:
    title=i.text.split('\n')[5]
    source.append(title)

In [461]:
source

['LVMH',
 'Tesla, SpaceX',
 'Amazon',
 'Oracle',
 'Berkshire Hathaway',
 'Microsoft',
 'Bloomberg LP',
 'Telecom',
 'Diversified',
 'Microsoft',
 "L'Oréal",
 'Google',
 'Zara',
 'Google',
 'Beverages, pharmaceuticals',
 'Facebook',
 'Koch Industries',
 'Koch Industries',
 'Walmart',
 'Walmart',
 'Walmart',
 'Media',
 'Dell Technologies',
 'Infrastructure, commodities',
 'Nike',
 'TikTok',
 'Retail',
 'Luxury goods',
 'Shipping',
 'Nutella, chocolates',
 'Candy, pet food',
 'Candy, pet food',
 'Diversified',
 'Internet media',
 'Casinos',
 'Hedge funds',
 'Red Bull',
 'Batteries',
 'Fashion retail',
 'Music, chemicals',
 'Chanel',
 'Chanel',
 'Shipping',
 'Shipping',
 'E-commerce',
 'Fasteners',
 'Real estate',
 'Trading, investments',
 'Hedge funds',
 'Investments',
 'BMW, pharmaceuticals',
 'Mining',
 'Online games',
 'Mining',
 'Software services',
 'Coal',
 'Discount brokerage',
 'Fertilizers, coal',
 'BMW',
 'Amazon',
 'Banking, tobacco',
 'Metals',
 'E-commerce',
 'Home appliances

In [462]:
industry=[]
industry.append(first_industry.text)
title_tags=driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]')
for i in title_tags:
    title=i.text.split('\n')[6]
    industry.append(title)

In [463]:
industry

['Fashion & Retail',
 'Automotive',
 'Technology',
 'Technology',
 'Finance & Investments',
 'Technology',
 'Media & Entertainment',
 'Telecom',
 'Diversified',
 'Technology',
 'Fashion & Retail',
 'Technology',
 'Fashion & Retail',
 'Technology',
 'Food & Beverage',
 'Technology',
 'Diversified',
 'Diversified',
 'Fashion & Retail',
 'Fashion & Retail',
 'Fashion & Retail',
 'Media & Entertainment',
 'Technology',
 'Diversified',
 'Fashion & Retail',
 'Technology',
 'Fashion & Retail',
 'Fashion & Retail',
 'Logistics',
 'Food & Beverage',
 'Food & Beverage',
 'Food & Beverage',
 'Diversified',
 'Technology',
 'Gambling & Casinos',
 'Finance & Investments',
 'Food & Beverage',
 'Automotive',
 'Fashion & Retail',
 'Diversified',
 'Fashion & Retail',
 'Fashion & Retail',
 'Logistics',
 'Logistics',
 'Technology',
 'Manufacturing',
 'Real Estate',
 'Finance & Investments',
 'Finance & Investments',
 'Finance & Investments',
 'Automotive',
 'Metals & Mining',
 'Technology',
 'Metals & Min

In [468]:
import pandas as pd
df=pd.DataFrame({'Rank':rank,'Name':name,'Net worth':net_worth,'Age':age,'Country':country,'Source':source,'Industry':industry})
df

,Rank,Name,Net worth,Age,Country,Source,Industry
0,1,Bernard Arnault & family,$211 B,74,France,LVMH,Fashion & Retail
1,2,Elon Musk,$180 B,51,United States,"Tesla, SpaceX",Automotive
2,3,Jeff Bezos,$114 B,59,United States,Amazon,Technology
3,4,Larry Ellison,$107 B,78,United States,Oracle,Technology
4,5,Warren Buffett,$106 B,92,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,195,Jin Baofang,$9.6 B,70,China,Solar panels,Energy
196,195,Luo Liguo & family,$9.6 B,67,China,Chemicals,Manufacturing
197,195,Marijke Mars,$9.6 B,58,United States,"Candy, pet food",Food & Beverage
198,195,Pamela Mars,$9.6 B,62,United States,"Candy, pet food",Food & Beverage


Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall
reviews, privates from price, dorms from price, facilities and property description. 

In [475]:
driver=webdriver.Chrome()

In [476]:
driver.get("https://www.hostelworld.com/")

In [479]:

location=driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div[2]/div/div[1]/div/div[1]/div/div/div[2]/input')
location.send_keys('London')

In [481]:
search=driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div[2]/div/div[1]/div/div[2]/div/ul/li[2]/button/div[2]/div[1]')
search.click()

In [482]:
nxt_btn=driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div[3]/div/div/button[2]')
nxt_btn.click()

In [483]:
srch_btn=driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div[3]/div/div/button')
srch_btn.click()

In [556]:
name=[]
title_tags=driver.find_elements(By.XPATH,'//div[@class="property-name"]')
for i in title_tags:
    title=i.text
    name.append(title)
    

In [558]:
name.pop(0)
name

["Wombat's City Hostel London",
 'Onefam Notting Hill by Hostel One',
 "St Christopher's Village",
 'Generator London',
 'Urbany Hostel London',
 'Safestay London Elephant & Castle',
 'Phoenix Hostel',
 'Safestay London Kensington Holland Park',
 'No.8 Willesden Hostel London',
 'London Backpackers',
 'No.8 Seven Sisters',
 'International Students House',
 'Queen Elizabeth Chelsea',
 'Barmy Badger Backpackers',
 "St Christopher's Hammersmith",
 'Saint James Backpackers',
 'YHA London Oxford Street',
 "St Christopher's Greenwich",
 "St Christopher's Camden",
 'Smart Camden Inn Hostel',
 'PubLove @ The Steam Engine, Waterloo',
 'Bell House Hostel',
 "St Christopher's Shepherds Bush",
 'Park Villa',
 'Book a Bed Hostels',
 'Dover Castle Hostel and Bar',
 'The Backpackshack',
 'Hostelle - women only hostel London',
 'Selina Camden',
 'Strand Continental']

In [486]:
rating=[]
title_tags=driver.find_elements(By.XPATH,'//span[@class="number"]')
for i in title_tags:
    title=i.text
    rating.append(title)

In [559]:
rating.pop(0)
rating

['9.0',
 '9.7',
 '8.0',
 '7.6',
 '9.4',
 '7.2',
 '8.0',
 '7.0',
 '6.4',
 '7.7',
 '5.2',
 '9.0',
 '7.4',
 '9.1',
 '7.4',
 '7.7',
 '8.7',
 '7.7',
 '6.9',
 '7.8',
 '8.2',
 '7.5',
 '7.3',
 '8.4',
 '7.3',
 '7.7',
 '8.6',
 '8.8',
 '8.7',
 '6.2']

In [524]:
dist=[]
title_tags=driver.find_elements(By.XPATH,'//span[@class="distance-description"]')
for i in title_tags:
    title=i.text
    dist.append(title)

In [525]:
dist

['- 3.6km from city centre',
 '- 5.5km from city centre',
 '- 1.8km from city centre',
 '- 3km from city centre',
 '- 5.4km from city centre',
 '- 1.7km from city centre',
 '- 4.2km from city centre',
 '- 5.8km from city centre',
 '- 10km from city centre',
 '- 11.9km from city centre',
 '- 9km from city centre',
 '- 3.3km from city centre',
 '- 5.7km from city centre',
 '- 5.5km from city centre',
 '- 7.5km from city centre',
 '- 5.5km from city centre',
 '- 2.1km from city centre',
 '- 7.6km from city centre',
 '- 4.3km from city centre',
 '- 4.4km from city centre',
 '- 0.5km from city centre',
 '- 4.2km from city centre',
 '- 7km from city centre',
 '- 6.3km from city centre',
 '- 6.9km from city centre',
 '- 1.7km from city centre',
 '- 10.9km from city centre',
 '- 5.1km from city centre',
 '- 5.5km from city centre',
 '- 1.3km from city centre']

In [637]:
pvt_price=[]
title_tags=driver.find_elements(By.XPATH,'//div[@class="property-accommodation-prices"]')
for i in title_tags:
    if i.text is None :
        pvt_price.append("--") 
    else:
        pvt_price.append(i.text)

In [638]:

pvt_price

['-20%\nPrivates From\n₹20,552\n₹16,441\n-20%\nDorms From\n₹4,876\n₹3,901',
 'Privates From\n₹12,173\nNo Dorms Available',
 '-10%\nPrivates From\n₹11,047\n₹9,943\n-10%\nDorms From\n₹3,366\n₹3,029',
 'Privates From\n₹14,169\nDorms From\n₹2,599',
 'Privates From\n₹15,668\nDorms From\n₹4,380',
 'Privates From\n₹16,450\nDorms From\n₹2,597',
 'No Privates Available\n-5%\nDorms From\n₹2,561\n₹2,433',
 'No Privates Available\nDorms From\n₹2,493',
 'No Privates Available\nDorms From\n₹1,642',
 'No Privates Available\nDorms From\n₹2,460',
 'No Privates Available\nDorms From\n₹1,852',
 'Privates From\n₹6,938\nNo Dorms Available',
 'No Privates Available\nDorms From\n₹2,353',
 'Privates From\n₹10,162\nNo Dorms Available',
 'No Privates Available\n-10%\nDorms From\n₹2,827\n₹2,545',
 'Privates From\n₹8,826\nDorms From\n₹3,504',
 'Privates From\n₹14,367\nNo Dorms Available',
 'No Privates Available\n-10%\nDorms From\n₹2,827\n₹2,545',
 'No Privates Available\n-10%\nDorms From\n₹2,808\n₹2,527',
 'No P

In [647]:
dorm_price=[]
title_tags=driver.find_elements(By.XPATH,'//div[@class="property-accommodation-prices"]')
for i in title_tags:
    if i.text is None :
        dorm_price.append("--") 
    else:
        dorm_price.append(i.text)

In [648]:
dorm_price

['-20%\nPrivates From\n₹20,552\n₹16,441\n-20%\nDorms From\n₹4,876\n₹3,901',
 'Privates From\n₹12,173\nNo Dorms Available',
 '-10%\nPrivates From\n₹11,047\n₹9,943\n-10%\nDorms From\n₹3,366\n₹3,029',
 'Privates From\n₹14,169\nDorms From\n₹2,599',
 'Privates From\n₹15,668\nDorms From\n₹4,380',
 'Privates From\n₹16,450\nDorms From\n₹2,597',
 'No Privates Available\n-5%\nDorms From\n₹2,561\n₹2,433',
 'No Privates Available\nDorms From\n₹2,493',
 'No Privates Available\nDorms From\n₹1,642',
 'No Privates Available\nDorms From\n₹2,460',
 'No Privates Available\nDorms From\n₹1,852',
 'Privates From\n₹6,938\nNo Dorms Available',
 'No Privates Available\nDorms From\n₹2,353',
 'Privates From\n₹10,162\nNo Dorms Available',
 'No Privates Available\n-10%\nDorms From\n₹2,827\n₹2,545',
 'Privates From\n₹8,826\nDorms From\n₹3,504',
 'Privates From\n₹14,367\nNo Dorms Available',
 'No Privates Available\n-10%\nDorms From\n₹2,827\n₹2,545',
 'No Privates Available\n-10%\nDorms From\n₹2,808\n₹2,527',
 'No P

In [649]:
review=[]
title_tags=driver.find_elements(By.XPATH,'//div[@class="review"]')
for i in title_tags:
    title=i.text
    review.append(title)

In [650]:
review

['(15114)',
 '(2157)',
 '(12278)',
 '(7620)',
 '(840)',
 '(5032)',
 '(4295)',
 '(1597)',
 '(5049)',
 '(4492)',
 '(4040)',
 '(1054)',
 '(3531)',
 '(1952)',
 '(4271)',
 '(1908)',
 '(4640)',
 '(3373)',
 '(4063)',
 '(3077)',
 '(404)',
 '(78)',
 '(784)',
 '(909)',
 '(1269)',
 '(2396)',
 '(768)',
 '(18)',
 '(54)',
 '(976)']

In [651]:
f_review=[]
title_tags=driver.find_elements(By.XPATH,'//span[@class="keyword"]')
for i in title_tags:
    title=i.text
    f_review.append(title)

In [653]:
f_review

['Fabulous',
 'Superb',
 'Superb',
 'Fabulous',
 'Very Good',
 'Superb',
 'Very Good',
 'Fabulous',
 'Very Good',
 'Good',
 'Superb',
 'Very Good',
 'Superb',
 'Very Good',
 'Very Good',
 'Fabulous',
 'Very Good',
 'Good',
 'Fabulous',
 'Very Good',
 'Very Good',
 'Fabulous',
 'Very Good',
 'Very Good',
 'Fabulous',
 'Fabulous',
 'Fabulous',
 'Good',
 'Very Good',
 'Very Good']

In [656]:
import pandas as pd
df=pd.DataFrame({'Name':name,'Rating':rating,'Distance':dist,'Private price':pvt_price,'Dorm Price':dorm_price,'Review':review,'Full review':f_review})
df

,Name,Rating,Distance,Private price,Dorm Price,Review,Full review
0,Wombat's City Hostel London,9.0,- 3.6km from city centre,"-20%\nPrivates From\n₹20,552\n₹16,441\n-20%\nD...","-20%\nPrivates From\n₹20,552\n₹16,441\n-20%\nD...",(15114),Fabulous
1,Onefam Notting Hill by Hostel One,9.7,- 5.5km from city centre,"Privates From\n₹12,173\nNo Dorms Available","Privates From\n₹12,173\nNo Dorms Available",(2157),Superb
2,St Christopher's Village,8.0,- 1.8km from city centre,"-10%\nPrivates From\n₹11,047\n₹9,943\n-10%\nDo...","-10%\nPrivates From\n₹11,047\n₹9,943\n-10%\nDo...",(12278),Superb
3,Generator London,7.6,- 3km from city centre,"Privates From\n₹14,169\nDorms From\n₹2,599","Privates From\n₹14,169\nDorms From\n₹2,599",(7620),Fabulous
4,Urbany Hostel London,9.4,- 5.4km from city centre,"Privates From\n₹15,668\nDorms From\n₹4,380","Privates From\n₹15,668\nDorms From\n₹4,380",(840),Very Good
5,Safestay London Elephant & Castle,7.2,- 1.7km from city centre,"Privates From\n₹16,450\nDorms From\n₹2,597","Privates From\n₹16,450\nDorms From\n₹2,597",(5032),Superb
6,Phoenix Hostel,8.0,- 4.2km from city centre,"No Privates Available\n-5%\nDorms From\n₹2,561...","No Privates Available\n-5%\nDorms From\n₹2,561...",(4295),Very Good
7,Safestay London Kensington Holland Park,7.0,- 5.8km from city centre,"No Privates Available\nDorms From\n₹2,493","No Privates Available\nDorms From\n₹2,493",(1597),Fabulous
8,No.8 Willesden Hostel London,6.4,- 10km from city centre,"No Privates Available\nDorms From\n₹1,642","No Privates Available\nDorms From\n₹1,642",(5049),Very Good
9,London Backpackers,7.7,- 11.9km from city centre,"No Privates Available\nDorms From\n₹2,460","No Privates Available\nDorms From\n₹2,460",(4492),Good


Write a program to scrap all the available details of best gaming laptops from digit.in.

In [657]:
driver=webdriver.Chrome()

In [660]:
driver.get('https://www.digit.in/')

In [661]:
search=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[2]')
search.click()

In [664]:
gaming_btn=driver.find_element(By.XPATH,'/html/body/div[3]/div/ul/li[5]/span')
gaming_btn.click()

In [665]:
srch_btn=driver.find_element(By.XPATH,'/html/body/div[3]/div/ul/li[5]/div[2]/div/div[2]/div/ul[2]/li[2]/a')
srch_btn.click()

In [667]:
bst_laptop=driver.find_element(By.XPATH,'/html/body/div[3]/div/div[1]/div[3]/div/div[3]/div[1]/a/h4')
bst_laptop.click()

In [669]:
b_name=[]
title_tags=driver.find_elements(By.XPATH,'//span[@class="datahreflink"]')
for i in title_tags:
    title=i.text
    b_name.append(title)


In [676]:
b_name
        

['Lenovo IdeaPad Gaming 3',
 'Acer Aspire 7',
 'Lenovo IdeaPad Gaming 3',
 'HP Pavilion Gaming 15',
 'Acer Aspire 7',
 'Lenovo IdeaPad Gaming 3',
 'MSI Bravo 15']

In [681]:
os=[]
title_tags=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]')
for i in title_tags:
    title=i.text.split(":")[1].split('\n')[0]
    os.append(title)

In [682]:
os

[' Windows 11 Home',
 ' Windows 11 Home',
 ' Windows 11 Home',
 ' Windows 10 Home',
 ' Windows 11 Home',
 ' Windows 11 Home',
 ' Windows 11 Home']

In [685]:
display=[]
title_tags=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]')
for i in title_tags:
    title=i.text.split(":")[2].split("\n")[0]
    display.append(title)

In [686]:
display

[' 15.6" (1920 x 1080)',
 ' 15.6" (1920 x 1080)',
 ' 15.6" (1920 x 1080)',
 ' 15.6" (1920 x 1080)',
 ' 15.6" (1920 x 1080)',
 ' 15.6" (1920 x 1080)',
 ' 15.6" (1920 x 1080)']

In [687]:
processor=[]
title_tags=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]')
for i in title_tags:
    title=i.text.split(":")[3].split('\n')[0]
    processor.append(title)

In [688]:
processor

[' AMD Ryzen 5-5600H | 3.3 GHz',
 ' AMD Ryzen 5-5500U | 2.1 GHz',
 ' 11th Gen Intel Core i5-11320H | 3.2 GHz',
 ' 5th Gen AMD Ryzen 5 5600H | 4.2 GHz',
 ' 12th Gen Intel Core i5-1240P | 3.30 GHz',
 ' AMD Ryzen 5-5600H | 3.2 GHz',
 ' AMD Ryzen 5-5600H | 3.3 GHz']

In [689]:
memory=[]
title_tags=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]')
for i in title_tags:
    title=i.text.split(":")[4].split('\n')[0]
    memory.append(title)

In [690]:
memory

[' 512 GB SSD/8 GB DDR4',
 ' 512 GB SSD/16 GB DDR4',
 ' 512 GB SSD/8 GB DDR4',
 ' 512 GB SSD/8 GB DDR4',
 ' 512 GB SSD/8 GB DDR4',
 ' 256 GB SSD/8 GB DDR4',
 ' 512 GB SSD/8 GB DDR4']

In [691]:
price=[]
title_tags=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]')
for i in title_tags:
    title=i.text.split(":")[5].split('\n')[0]
    price.append(title)

In [692]:
price

[' ₹ 54,490',
 ' ₹ 49,990',
 ' ₹ 57,890',
 ' ₹ 60,490',
 ' ₹ 52,990',
 ' ₹ 49,990',
 ' ₹ 47,990']

In [693]:
import pandas as pd
df=pd.DataFrame({'Brand':b_name,'Operating system':os,'Processor':processor,'Memory':memory,'Display':display,'Price':price})
df

,Brand,Operating system,Processor,Memory,Display,Price
0,Lenovo IdeaPad Gaming 3,Windows 11 Home,AMD Ryzen 5-5600H | 3.3 GHz,512 GB SSD/8 GB DDR4,"15.6"" (1920 x 1080)","₹ 54,490"
1,Acer Aspire 7,Windows 11 Home,AMD Ryzen 5-5500U | 2.1 GHz,512 GB SSD/16 GB DDR4,"15.6"" (1920 x 1080)","₹ 49,990"
2,Lenovo IdeaPad Gaming 3,Windows 11 Home,11th Gen Intel Core i5-11320H | 3.2 GHz,512 GB SSD/8 GB DDR4,"15.6"" (1920 x 1080)","₹ 57,890"
3,HP Pavilion Gaming 15,Windows 10 Home,5th Gen AMD Ryzen 5 5600H | 4.2 GHz,512 GB SSD/8 GB DDR4,"15.6"" (1920 x 1080)","₹ 60,490"
4,Acer Aspire 7,Windows 11 Home,12th Gen Intel Core i5-1240P | 3.30 GHz,512 GB SSD/8 GB DDR4,"15.6"" (1920 x 1080)","₹ 52,990"
5,Lenovo IdeaPad Gaming 3,Windows 11 Home,AMD Ryzen 5-5600H | 3.2 GHz,256 GB SSD/8 GB DDR4,"15.6"" (1920 x 1080)","₹ 49,990"
6,MSI Bravo 15,Windows 11 Home,AMD Ryzen 5-5600H | 3.3 GHz,512 GB SSD/8 GB DDR4,"15.6"" (1920 x 1080)","₹ 47,990"


4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com
and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand
Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the
details is missing then replace it by “- “. Save your results in a dataframe and CSV

In [713]:
driver=webdriver.Chrome()


In [714]:
driver.get('https://www.flipkart.com/')

In [715]:
search=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')
search.send_keys('smartphone')

In [716]:
srch_btn=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button')
srch_btn.click()

In [720]:
b_name=[]
title_tags=driver.find_elements(By.XPATH,"//div[@class='_4rR01T']")
for i in title_tags:
    if i.text is None :
        b_name.append("--") 
    else:
        b_name.append(i.text)


In [721]:
b_name

['SAMSUNG Galaxy F13 (Waterfall Blue, 64 GB)',
 'SAMSUNG Galaxy F13 (Nightsky Green, 64 GB)',
 'SAMSUNG Galaxy F13 (Sunrise Copper, 64 GB)',
 'SAMSUNG Galaxy F04 (Jade Purple, 64 GB)',
 'APPLE iPhone 13 (Starlight, 128 GB)',
 'POCO C51 (Royal Blue, 64 GB)',
 'POCO C55 (Forest Green, 128 GB)',
 'POCO C55 (Power Black, 128 GB)',
 'APPLE iPhone 13 (Midnight, 128 GB)',
 'POCO C51 (Power Black, 64 GB)',
 'APPLE iPhone 13 (Green, 128 GB)',
 'realme C53 (Champion Gold, 64 GB)',
 'APPLE iPhone 13 (Pink, 128 GB)',
 'realme C53 (Champion Black, 64 GB)',
 'Infinix SMART 7 (Azure Blue, 64 GB)',
 'POCO C50 (Royal Blue, 32 GB)',
 'OPPO A17 (Lake Blue, 64 GB)',
 'Infinix Smart 7 HD (Green Apple, 64 GB)',
 'Infinix Smart 7 HD (Silk Blue, 64 GB)',
 'Infinix Smart 7 HD (Jade White, 64 GB)',
 'Infinix Smart 7 HD (Ink Black, 64 GB)',
 'OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)',
 'Infinix SMART 7 (Emerald Green, 64 GB)',
 'POCO C50 (Country Green, 32 GB)']

In [722]:
Ram_rom=[]
title_tags=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[1]")
for i in title_tags:
    if i.text is None :
        Ram_rom.append("--") 
    else:
        Ram_rom.append(i.text)

In [723]:
Ram_rom

['4 GB RAM | 64 GB ROM | Expandable Upto 1 TB',
 '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB',
 '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB',
 '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB',
 '128 GB ROM',
 '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB',
 '6 GB RAM | 128 GB ROM | Expandable Upto 1 TB',
 '6 GB RAM | 128 GB ROM | Expandable Upto 1 TB',
 '128 GB ROM',
 '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB',
 '128 GB ROM',
 '6 GB RAM | 64 GB ROM | Expandable Upto 2 TB',
 '128 GB ROM',
 '6 GB RAM | 64 GB ROM | Expandable Upto 2 TB',
 '4 GB RAM | 64 GB ROM | Expandable Upto 2 TB',
 '2 GB RAM | 32 GB ROM | Expandable Upto 512 GB',
 '4 GB RAM | 64 GB ROM | Expandable Upto 2 TB',
 '2 GB RAM | 64 GB ROM | Expandable Upto 1 TB',
 '2 GB RAM | 64 GB ROM | Expandable Upto 1 TB',
 '2 GB RAM | 64 GB ROM | Expandable Upto 1 TB',
 '2 GB RAM | 64 GB ROM | Expandable Upto 1 TB',
 '6 GB RAM | 128 GB ROM',
 '4 GB RAM | 64 GB ROM | Expandable Upto 2 TB',
 '2 GB RAM | 32 GB ROM | Expandable Upto 512 G

In [724]:
camera=[]
title_tags=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[3]")
for i in title_tags:
    if i.text is None :
        camera.append("--") 
    else:
        camera.append(i.text)


In [725]:
camera

['50MP + 5MP + 2MP | 8MP Front Camera',
 '50MP + 5MP + 2MP | 8MP Front Camera',
 '50MP + 5MP + 2MP | 8MP Front Camera',
 '13MP + 2MP | 5MP Front Camera',
 '12MP + 12MP | 12MP Front Camera',
 '8MP Dual Rear Camera | 5MP Front Camera',
 '50MP Dual Rear Camera | 5MP Front Camera',
 '50MP Dual Rear Camera | 5MP Front Camera',
 '12MP + 12MP | 12MP Front Camera',
 '8MP Dual Rear Camera | 5MP Front Camera',
 '12MP + 12MP | 12MP Front Camera',
 '108MP + 2MP | 8MP Front Camera',
 '12MP + 12MP | 12MP Front Camera',
 '108MP + 2MP | 8MP Front Camera',
 '13MP + AI Lens | 5MP Front Camera',
 '8MP Dual Camera | 5MP Front Camera',
 '50MP + 0.3MP + 0.3MP | 5MP Front Camera',
 '8MP + AI Lens | 5MP Front Camera',
 '8MP + AI Lens | 5MP Front Camera',
 '8MP + AI Lens | 5MP Front Camera',
 '8MP + AI Lens | 5MP Front Camera',
 '64MP Rear Camera | 16MP Front Camera',
 '13MP + AI Lens | 5MP Front Camera',
 '8MP Dual Camera | 5MP Front Camera']

In [726]:
display=[]
title_tags=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[2]")
for i in title_tags:
    if i.text is None :
        display.append("--") 
    else:
        display.append(i.text)

In [727]:
display

['16.76 cm (6.6 inch) Full HD+ Display',
 '16.76 cm (6.6 inch) Full HD+ Display',
 '16.76 cm (6.6 inch) Full HD+ Display',
 '16.51 cm (6.5 inch) HD Display',
 '15.49 cm (6.1 inch) Super Retina XDR Display',
 '16.56 cm (6.52 inch) HD+ Display',
 '17.04 cm (6.71 inch) HD+ Display',
 '17.04 cm (6.71 inch) HD+ Display',
 '15.49 cm (6.1 inch) Super Retina XDR Display',
 '16.56 cm (6.52 inch) HD+ Display',
 '15.49 cm (6.1 inch) Super Retina XDR Display',
 '17.12 cm (6.74 inch) HD Display',
 '15.49 cm (6.1 inch) Super Retina XDR Display',
 '17.12 cm (6.74 inch) HD Display',
 '16.76 cm (6.6 inch) HD+ Display',
 '16.56 cm (6.52 inch) HD+ Display',
 '16.66 cm (6.56 inch) HD+ Display',
 '16.76 cm (6.6 inch) HD+ Display',
 '16.76 cm (6.6 inch) HD+ Display',
 '16.76 cm (6.6 inch) HD+ Display',
 '16.76 cm (6.6 inch) HD+ Display',
 '16.74 cm (6.59 inch) Display',
 '16.76 cm (6.6 inch) HD+ Display',
 '16.56 cm (6.52 inch) HD+ Display']

In [728]:
processor=[]
title_tags=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[5]")
for i in title_tags:
    if i.text is None :
        processor.append("--") 
    else:
        processor.append(i.text)

In [729]:
processor

['Exynos 850 Processor',
 'Exynos 850 Processor',
 'Exynos 850 Processor',
 'Mediatek Helio P35 Processor',
 'Brand Warranty for 1 Year',
 'Helio G36 Processor',
 'Mediatek Helio G85 Processor',
 'Mediatek Helio G85 Processor',
 'Brand Warranty for 1 Year',
 'Helio G36 Processor',
 'Brand Warranty for 1 Year',
 'T612 Processor',
 'Brand Warranty for 1 Year',
 'T612 Processor',
 'Unisoc Spreadtrum SC9863A1 Processor',
 'Mediatek Helio A22 Processor, Upto 2.0 GHz Processor',
 'Mediatek Helio G35 Processor',
 'Spreadtrum SC9863A1 Processor',
 'Spreadtrum SC9863A1 Processor',
 'Spreadtrum SC9863A1 Processor',
 'Spreadtrum SC9863A1 Processor',
 '12 Months',
 'Unisoc Spreadtrum SC9863A1 Processor',
 'Mediatek Helio A22 Processor, Upto 2.0 GHz Processor']

In [730]:
price=[]
title_tags=driver.find_elements(By.XPATH,"//div[@class='_30jeq3 _1_WHN1']")
for i in title_tags:
    if i.text is None :
        price.append("--") 
    else:
        price.append(i.text)

In [731]:
price

['₹9,599',
 '₹9,599',
 '₹9,599',
 '₹7,099',
 '₹58,499',
 '₹6,499',
 '₹8,499',
 '₹8,499',
 '₹58,499',
 '₹6,499',
 '₹58,499',
 '₹10,999',
 '₹58,499',
 '₹10,999',
 '₹6,999',
 '₹5,499',
 '₹12,499',
 '₹5,999',
 '₹5,999',
 '₹5,999',
 '₹5,999',
 '₹17,900',
 '₹6,999',
 '₹5,499']

In [732]:
battery=[]
title_tags=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[4]")
for i in title_tags:
    if i.text is None :
        battery.append("--") 
    else:
        battery.append(i.text)

In [733]:
battery

['6000 mAh Lithium Ion Battery',
 '6000 mAh Lithium Ion Battery',
 '6000 mAh Lithium Ion Battery',
 '5000 mAh Lithium-Ion Battery',
 'A15 Bionic Chip Processor',
 '5000 mAh Battery',
 '5000 mAh Battery',
 '5000 mAh Battery',
 'A15 Bionic Chip Processor',
 '5000 mAh Battery',
 'A15 Bionic Chip Processor',
 '5000 mAh Battery',
 'A15 Bionic Chip Processor',
 '5000 mAh Battery',
 '6000 mAh Battery',
 '5000 mAh Lithium Polymer Battery',
 '5000 mAh Lithium-ion Polymer Battery',
 '5000 mAh Battery',
 '5000 mAh Battery',
 '5000 mAh Battery',
 '5000 mAh Battery',
 '5000 mAh Battery',
 '6000 mAh Battery',
 '5000 mAh Lithium Polymer Battery']

In [734]:
import pandas as pd
df=pd.DataFrame({'Brand':b_name,'Price':price,'Camera':camera,'Memory':Ram_rom,'Battery':battery,'Processor':processor,'Display':display})
df

,Brand,Price,Camera,Memory,Battery,Processor,Display
0,"SAMSUNG Galaxy F13 (Waterfall Blue, 64 GB)","₹9,599",50MP + 5MP + 2MP | 8MP Front Camera,4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,6000 mAh Lithium Ion Battery,Exynos 850 Processor,16.76 cm (6.6 inch) Full HD+ Display
1,"SAMSUNG Galaxy F13 (Nightsky Green, 64 GB)","₹9,599",50MP + 5MP + 2MP | 8MP Front Camera,4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,6000 mAh Lithium Ion Battery,Exynos 850 Processor,16.76 cm (6.6 inch) Full HD+ Display
2,"SAMSUNG Galaxy F13 (Sunrise Copper, 64 GB)","₹9,599",50MP + 5MP + 2MP | 8MP Front Camera,4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,6000 mAh Lithium Ion Battery,Exynos 850 Processor,16.76 cm (6.6 inch) Full HD+ Display
3,"SAMSUNG Galaxy F04 (Jade Purple, 64 GB)","₹7,099",13MP + 2MP | 5MP Front Camera,4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,5000 mAh Lithium-Ion Battery,Mediatek Helio P35 Processor,16.51 cm (6.5 inch) HD Display
4,"APPLE iPhone 13 (Starlight, 128 GB)","₹58,499",12MP + 12MP | 12MP Front Camera,128 GB ROM,A15 Bionic Chip Processor,Brand Warranty for 1 Year,15.49 cm (6.1 inch) Super Retina XDR Display
5,"POCO C51 (Royal Blue, 64 GB)","₹6,499",8MP Dual Rear Camera | 5MP Front Camera,4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,5000 mAh Battery,Helio G36 Processor,16.56 cm (6.52 inch) HD+ Display
6,"POCO C55 (Forest Green, 128 GB)","₹8,499",50MP Dual Rear Camera | 5MP Front Camera,6 GB RAM | 128 GB ROM | Expandable Upto 1 TB,5000 mAh Battery,Mediatek Helio G85 Processor,17.04 cm (6.71 inch) HD+ Display
7,"POCO C55 (Power Black, 128 GB)","₹8,499",50MP Dual Rear Camera | 5MP Front Camera,6 GB RAM | 128 GB ROM | Expandable Upto 1 TB,5000 mAh Battery,Mediatek Helio G85 Processor,17.04 cm (6.71 inch) HD+ Display
8,"APPLE iPhone 13 (Midnight, 128 GB)","₹58,499",12MP + 12MP | 12MP Front Camera,128 GB ROM,A15 Bionic Chip Processor,Brand Warranty for 1 Year,15.49 cm (6.1 inch) Super Retina XDR Display
9,"POCO C51 (Power Black, 64 GB)","₹6,499",8MP Dual Rear Camera | 5MP Front Camera,4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,5000 mAh Battery,Helio G36 Processor,16.56 cm (6.52 inch) HD+ Display


In [61]:
#saving dataframe to csv file
df.to_csv('brand_scraping.csv')